In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path().absolute().parent))

In [2]:
import ee 
import geemap

ee.Initialize(project="thurgau-irrigation")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_0JLhFqfSY1uiEaW?source=Init


In [3]:
from src.et_blue.exporting_utils import process_et_blue
from src.et_blue.compute_et_blue import compute_et_blue

from utils.ee_utils import (
    back_to_float,
    back_to_int,
    export_image_to_asset,
    print_value_ranges,
)

from utils.date_utils import print_collection_dates

---

## Constants

In [4]:
YEAR = 2023

PATH_TO_AOI = "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
PATH_TO_ET_PRODUCT = (
    f"projects/thurgau-irrigation/assets/ET_products/Thurgau/WaPOR_10m_{YEAR}"
)
PATHT_TO_ET_GREEN = f"projects/thurgau-irrigation/assets/Thurgau/ET_green/ET_green_{YEAR}_dekadal_from_WaPOR_10m"

ET_GREEN_BAND_NAME = "ET_green"
ET_BAND_NAME = "downscaled"
SCALING_FACTOR = 100  # scaling factor for the ET band to convert it back to float
# DYNAMIC = False
# SCALING_FACTOR_PROPERTY_NAME = "days" if DYNAMIC else None
NUMBER_OF_IMAGES = 36
TEMPORAL_RESOLUTION = "dekadal"


ET_BLUE_RAW_PATH = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_{YEAR}_{TEMPORAL_RESOLUTION}_from_WaPOR_10m"
EXPORT_IMAGE_RESOLUTION = 10  # in meters

## 1. Load Assets

In [5]:
aoi = ee.FeatureCollection(PATH_TO_AOI).geometry().simplify(500).buffer(100)

In [6]:
et_collection = (
    ee.ImageCollection(PATH_TO_ET_PRODUCT)
    # .filter(ee.Filter.eq("Region", "Thurgau"))
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31").map(
        lambda img: back_to_float(img, SCALING_FACTOR)
    )
).sort("system:time_start")

et_collection_list = et_collection.toList(et_collection.size())


# Sanity checks:
print_collection_dates(et_collection)

Dates of images in the collection:
2023-01-01
2023-01-11
2023-01-21
2023-02-01
2023-02-11
2023-02-21
2023-03-01
2023-03-11
2023-03-21
2023-04-01
2023-04-11
2023-04-21
2023-05-01
2023-05-11
2023-05-21
2023-06-01
2023-06-11
2023-06-21
2023-07-01
2023-07-11
2023-07-21
2023-08-01
2023-08-11
2023-08-21
2023-09-01
2023-09-11
2023-09-21
2023-10-01
2023-10-11
2023-10-21
2023-11-01
2023-11-11
2023-11-21
2023-12-01
2023-12-11
2023-12-21


In [7]:
et_green_collection = (
    ee.ImageCollection(PATHT_TO_ET_GREEN)
    .filterDate(f"{YEAR}-01-01", f"{YEAR}-12-31")
    .map(lambda img: back_to_float(img, SCALING_FACTOR))
)

et_green_list = et_green_collection.toList(et_collection.size())

# Sanity checks:
print_collection_dates(et_green_collection)
# print_value_ranges(et_green_collection, ET_GREEN_BAND_NAME)

Dates of images in the collection:
2023-01-01
2023-01-11
2023-01-21
2023-02-01
2023-02-11
2023-02-21
2023-03-01
2023-03-11
2023-03-21
2023-04-01
2023-04-11
2023-04-21
2023-05-01
2023-05-11
2023-05-21
2023-06-01
2023-06-11
2023-06-21
2023-07-01
2023-07-11
2023-07-21
2023-08-01
2023-08-11
2023-08-21
2023-09-01
2023-09-11
2023-09-21
2023-10-01
2023-10-11
2023-10-21
2023-11-01
2023-11-11
2023-11-21
2023-12-01
2023-12-11
2023-12-21


## 2. Compute and Export ET_blue raw

In [8]:
process_et_blue(
    et_collection_list=et_collection_list,
    et_green_list=et_green_list,
    year=YEAR,
    aoi=aoi,
    asset_path=ET_BLUE_RAW_PATH,
    time_step_type=TEMPORAL_RESOLUTION,
    resolution=EXPORT_IMAGE_RESOLUTION,
)

Exporting ET_blue_raw_dekadal_2023_01_D1 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_2023_dekadal_from_WaPOR_10m/ET_blue_raw_dekadal_2023_01_D1
Exporting ET_blue_raw_dekadal_2023_01_D2 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_2023_dekadal_from_WaPOR_10m/ET_blue_raw_dekadal_2023_01_D2
Exporting ET_blue_raw_dekadal_2023_01_D3 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_2023_dekadal_from_WaPOR_10m/ET_blue_raw_dekadal_2023_01_D3
Exporting ET_blue_raw_dekadal_2023_02_D1 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_2023_dekadal_from_WaPOR_10m/ET_blue_raw_dekadal_2023_02_D1
Exporting ET_blue_raw_dekadal_2023_02_D2 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw/ET_blue_raw_2023_dekadal_from_WaPOR_10m/ET_blue_raw_dekadal_2023_02_D2
Exporting ET_blue_raw_dekadal_2023_02_D3 for 2023 to projects/thurgau-irrigation/assets/Thurgau/ET_blue_r

In [9]:
et_blue_raw = []

for i in range(NUMBER_OF_IMAGES):
    # Process ET images
    et_image = ee.Image(et_collection_list.get(i))
    et_green = ee.Image(et_green_list.get(i))
    et_blue = compute_et_blue(et_image, et_green)
    et_blue_raw.append(et_blue)

In [10]:
ee.Image(et_blue_raw[0]).bandNames().getInfo()

['ET_blue']

In [11]:
Map = geemap.Map()

image = ee.Image(et_blue_raw[20])
# image_green = ee.Image(et_green_list.get(20))
# image_et = ee.Image(et_collection_list.get(20))
# image_ET = ee.Image(et_collection_list.get(17))

vis_params = {
    "bands": ["ET_blue"],
    "min": 0,
    "max": 1,
    "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
}

# vis_green = {
#     "bands": ["ET_green"],
#     "min": 0,
#     "max": 3,
#     "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
# }

# vis_ET = {
#     "bands": ["downscaled"],
#     "min": 0,
#     "max": 5,
#     "palette": ["blue", "lightblue", "green", "yellow", "orange", "red"],
# }


Map.center_object(aoi, 12)
Map.addLayer(image, vis_params, "ET blue raw")
Map.add_colorbar(vis_params, label="ET blue raw [mm/month]")
# Map.addLayer(image_green, vis_green, "ET green")
# Map.add_colorbar(vis_green, label="ET green [mm/month]")
# Map.addLayer(image_et, vis_ET, "ET")
# Map.add_colorbar(vis_ET, label="ET [mm/month]")


Map

Map(center=[47.56858787382066, 9.092720596553875], controls=(WidgetControl(options=['position', 'transparent_b…